<a href="https://colab.research.google.com/github/gomlfx/ApiaryFund/blob/master/XGBR1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#designed for Jupyter/kaggle/colab
import pandas as pd
import numpy as np
#import matplotlib for plotting 
import matplotlib.pyplot as plt
%matplotlib inline
#import normalization
from sklearn import preprocessing
import xgboost as xg
from sklearn.ensemble import RandomForestRegressor as rf

In [2]:
 #MT4 csv 
df1=pd.read_csv('EURUSD1440.csv', names=['d','t','eu_o','eu_h','eu_l','eu_c','eu_v'])
df1 = df1.tail(2000)
df2=pd.read_csv('GBPUSD1440.csv', names=['d','t','gu_o','gu_h','gu_l','gu_c','gu_v'])
df2 = df2.tail(2000)

df3=pd.read_csv('USDCAD1440.csv', names=['d','t','uc_o','uc_h','uc_l','uc_c','uc_v'])
df3 = df3.tail(2000)

df4=pd.read_csv('USDCHF1440.csv', names=['d','t','uf_o','uf_h','uf_l','uf_c','uf_v'])
df4 = df4.tail(2000)

df5=pd.read_csv('USDJPY1440.csv', names=['d','t','uj_o','uj_h','uj_l','uj_c','uj_v'])
df5 = df5.tail(2000)

df6=pd.read_csv('USDSEK1440.csv', names=['d','t','us_o','us_h','us_l','us_c','us_v'])
df6 = df6.tail(2000)

#using merge
df_m_1 = df1.merge(df2, on='d')
df_m_2 = df_m_1.merge(df3, on='d')
df_m_3 = df_m_2.merge(df4, on='d')
df_m_4 = df_m_3.merge(df5, on='d')
df_m_5 = df_m_4.merge(df6, on='d')
df_m_5 = df_m_5.drop(columns='t_x')
df_m_5 = df_m_5.drop(columns='t_y')

pd.set_option('display.max_columns', None)
print(df_m_5) 

               d     eu_o     eu_h     eu_l     eu_c    eu_v     gu_o  \
0     2014.03.14  1.38676  1.39374  1.38473  1.39134   77320  1.66229   
1     2014.03.17  1.39062  1.39474  1.38788  1.39252   54560  1.66419   
2     2014.03.18  1.39251  1.39427  1.38797  1.39304   65439  1.66356   
3     2014.03.19  1.39304  1.39329  1.38095  1.38223   71553  1.65916   
4     2014.03.20  1.38226  1.38448  1.37490  1.37782   73086  1.65376   
...          ...      ...      ...      ...      ...     ...      ...   
1993  2021.11.23  1.12339  1.12748  1.12258  1.12483  104536  1.33930   
1994  2021.11.24  1.12467  1.12551  1.11859  1.12000  112792  1.33744   
1995  2021.11.25  1.11959  1.12295  1.11959  1.12062   44916  1.33243   
1996  2021.11.26  1.12058  1.13295  1.12051  1.13177  138554  1.33158   
1997  2021.11.29  1.13063  1.13117  1.13052  1.13097     207  1.33386   

         gu_h     gu_l     gu_c   gu_v     uc_o     uc_h     uc_l     uc_c  \
0     1.66499  1.65869  1.66439  65960  1.107

In [3]:
df_m_5.tail()

,d,eu_o,eu_h,eu_l,eu_c,eu_v,gu_o,gu_h,gu_l,gu_c,gu_v,uc_o,uc_h,uc_l,uc_c,uc_v,uf_o,uf_h,uf_l,uf_c,uf_v,uj_o,uj_h,uj_l,uj_c,uj_v,us_o,us_h,us_l,us_c,us_v
1993,2021.11.23,1.12339,1.12748,1.12258,1.12483,104536,1.33930,1.34086,1.33418,1.33780,35730,1.26948,1.27444,1.26599,1.26693,34331,0.93086,0.93439,0.93011,0.93313,30715,114.870,115.192,114.480,115.129,49655,8.98692,9.05286,8.97004,9.04097,40131
1994,2021.11.24,1.12467,1.12551,1.11859,1.12000,112792,1.33744,1.33892,1.33151,1.33280,33435,1.26663,1.27097,1.26509,1.26648,34606,0.93283,0.93733,0.93209,0.93392,29908,115.111,115.521,114.822,115.401,47423,9.03311,9.14057,9.02853,9.12259,36605
1995,2021.11.25,1.11959,1.12295,1.11959,1.12062,44916,1.33243,1.33530,1.33047,1.33164,25719,1.26555,1.26766,1.26402,1.26456,19538,0.93382,0.93613,0.93239,0.93564,21596,115.402,115.455,115.239,115.341,23627,9.11333,9.12367,9.05598,9.07990,26093
1996,2021.11.26,1.12058,1.13295,1.12051,1.13177,138554,1.33158,1.33596,1.32772,1.33235,42921,1.26449,1.27994,1.26449,1.27860,41366,0.93575,0.93589,0.92107,0.92347,44008,115.296,115.365,113.052,113.285,87016,9.08432,9.18200,9.06362,9.14441,40881
1997,2021.11.29,1.13063,1.13117,1.13052,1.13097,207,1.33386,1.33386,1.33263,1.33288,100,1.27580,1.27616,1.27517,1.27557,315,0.92255,0.92295,0.92145,0.92238,47,113.538,113.578,113.354,113.380,679,9.11969,9.13061,9.11042,9.12102,108


In [4]:
df_m_5.insert(1,'index',df_m_5.index)

In [5]:
#new column: high vs low
df_m_5['eu_h_or_l'] = ((df_m_5.eu_h - df_m_5.eu_o) > (df_m_5.eu_o - df_m_5.eu_l)) 
df_m_5.eu_h_or_l = df_m_5.eu_h_or_l.replace({True:1,False:0})

#new column: close vs yesterday
df_m_5['eu_c_vs_c'] = (df_m_5.eu_c > df_m_5.eu_o) 
df_m_5.eu_c_vs_c = df_m_5.eu_c_vs_c.replace({True:1,False:0})

#new column: shift tomorrow to today
df_m_5['eu_High_next_day'] = df_m_5['eu_h'].shift(-1)
df_m_5['eu_Low_next_day'] = df_m_5['eu_l'].shift(-1)
df_m_5['eu_Close_next_day'] = df_m_5['eu_c'].shift(-1)
df_m_5 = df_m_5.dropna()


In [6]:
df_m_5.tail()

,d,index,eu_o,eu_h,eu_l,eu_c,eu_v,gu_o,gu_h,gu_l,gu_c,gu_v,uc_o,uc_h,uc_l,uc_c,uc_v,uf_o,uf_h,uf_l,uf_c,uf_v,uj_o,uj_h,uj_l,uj_c,uj_v,us_o,us_h,us_l,us_c,us_v,eu_h_or_l,eu_c_vs_c,eu_High_next_day,eu_Low_next_day,eu_Close_next_day
1992,2021.11.22,1992,1.12878,1.12903,1.12306,1.12333,110260,1.34503,1.34503,1.33838,1.33944,36466,1.26400,1.27048,1.26277,1.26995,28908,0.92767,0.93345,0.92667,0.93325,29297,113.937,114.959,113.923,114.871,48575,8.93010,9.01339,8.92970,8.99650,35881,0,0,1.12748,1.12258,1.12483
1993,2021.11.23,1993,1.12339,1.12748,1.12258,1.12483,104536,1.33930,1.34086,1.33418,1.33780,35730,1.26948,1.27444,1.26599,1.26693,34331,0.93086,0.93439,0.93011,0.93313,30715,114.870,115.192,114.480,115.129,49655,8.98692,9.05286,8.97004,9.04097,40131,1,1,1.12551,1.11859,1.12000
1994,2021.11.24,1994,1.12467,1.12551,1.11859,1.12000,112792,1.33744,1.33892,1.33151,1.33280,33435,1.26663,1.27097,1.26509,1.26648,34606,0.93283,0.93733,0.93209,0.93392,29908,115.111,115.521,114.822,115.401,47423,9.03311,9.14057,9.02853,9.12259,36605,0,0,1.12295,1.11959,1.12062
1995,2021.11.25,1995,1.11959,1.12295,1.11959,1.12062,44916,1.33243,1.33530,1.33047,1.33164,25719,1.26555,1.26766,1.26402,1.26456,19538,0.93382,0.93613,0.93239,0.93564,21596,115.402,115.455,115.239,115.341,23627,9.11333,9.12367,9.05598,9.07990,26093,1,1,1.13295,1.12051,1.13177
1996,2021.11.26,1996,1.12058,1.13295,1.12051,1.13177,138554,1.33158,1.33596,1.32772,1.33235,42921,1.26449,1.27994,1.26449,1.27860,41366,0.93575,0.93589,0.92107,0.92347,44008,115.296,115.365,113.052,113.285,87016,9.08432,9.18200,9.06362,9.14441,40881,1,1,1.13117,1.13052,1.13097


In [7]:

df_m_5.isna().sum()

d                    0
index                0
eu_o                 0
eu_h                 0
eu_l                 0
eu_c                 0
eu_v                 0
gu_o                 0
gu_h                 0
gu_l                 0
gu_c                 0
gu_v                 0
uc_o                 0
uc_h                 0
uc_l                 0
uc_c                 0
uc_v                 0
uf_o                 0
uf_h                 0
uf_l                 0
uf_c                 0
uf_v                 0
uj_o                 0
uj_h                 0
uj_l                 0
uj_c                 0
uj_v                 0
us_o                 0
us_h                 0
us_l                 0
us_c                 0
us_v                 0
eu_h_or_l            0
eu_c_vs_c            0
eu_High_next_day     0
eu_Low_next_day      0
eu_Close_next_day    0
dtype: int64

In [8]:
df_m_5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1997 entries, 0 to 1996
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   d                  1997 non-null   object 
 1   index              1997 non-null   int64  
 2   eu_o               1997 non-null   float64
 3   eu_h               1997 non-null   float64
 4   eu_l               1997 non-null   float64
 5   eu_c               1997 non-null   float64
 6   eu_v               1997 non-null   int64  
 7   gu_o               1997 non-null   float64
 8   gu_h               1997 non-null   float64
 9   gu_l               1997 non-null   float64
 10  gu_c               1997 non-null   float64
 11  gu_v               1997 non-null   int64  
 12  uc_o               1997 non-null   float64
 13  uc_h               1997 non-null   float64
 14  uc_l               1997 non-null   float64
 15  uc_c               1997 non-null   float64
 16  uc_v               1997 

In [9]:
#for high prediction
#select rows to use for x,y
x = df_m_5.iloc[:,np.r_[1:34]].values
print(x)
y = df_m_5.iloc[:,[34]].values
print(y) 

[[0.00000e+00 1.38676e+00 1.39374e+00 ... 6.36990e+04 1.00000e+00
  1.00000e+00]
 [1.00000e+00 1.39062e+00 1.39474e+00 ... 4.66600e+04 1.00000e+00
  1.00000e+00]
 [2.00000e+00 1.39251e+00 1.39427e+00 ... 5.52010e+04 0.00000e+00
  1.00000e+00]
 ...
 [1.99400e+03 1.12467e+00 1.12551e+00 ... 3.66050e+04 0.00000e+00
  0.00000e+00]
 [1.99500e+03 1.11959e+00 1.12295e+00 ... 2.60930e+04 1.00000e+00
  1.00000e+00]
 [1.99600e+03 1.12058e+00 1.13295e+00 ... 4.08810e+04 1.00000e+00
  1.00000e+00]]
[[1.39474]
 [1.39427]
 [1.39329]
 ...
 [1.12295]
 [1.13295]
 [1.13117]]


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)

In [11]:
#for high prediction and accuracy test
# create regressor object
regressor = xg.XGBRegressor(n_estimators=10000, objective='reg:squarederror', nthread=-1, n_jobs=-1)
#regressor = rf(n_estimators = 1000, n_jobs=-1, criterion="absolute_error", random_state=3)
# fit the regressor with x and y data
regressor.fit(X_train, y_train)

XGBRegressor(n_estimators=10000, n_jobs=-1, nthread=-1,
             objective='reg:squarederror')

In [12]:
y_pred = regressor.predict(X_test)

In [13]:
def mean_absolute_percentage_error(y_true,y_pred):
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_test,y_pred)

In [14]:
#for regression score
from sklearn import metrics 
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('MAPE:', mape)
print('MedAE', metrics.median_absolute_error(y_test,y_pred)) 

MAE: 0.0031118045354207344
MSE: 1.8170286544004365e-05
RMSE: 0.004262661908245171
MAPE: 6.344650968204937
MedAE 0.0025380512237549446


In [15]:
#extract row for variable
last_row = df_m_5.iloc[-1:,np.r_[1:34]].values
print(last_row) 

[[1.99600e+03 1.12058e+00 1.13295e+00 1.12051e+00 1.13177e+00 1.38554e+05
  1.33158e+00 1.33596e+00 1.32772e+00 1.33235e+00 4.29210e+04 1.26449e+00
  1.27994e+00 1.26449e+00 1.27860e+00 4.13660e+04 9.35750e-01 9.35890e-01
  9.21070e-01 9.23470e-01 4.40080e+04 1.15296e+02 1.15365e+02 1.13052e+02
  1.13285e+02 8.70160e+04 9.08432e+00 9.18200e+00 9.06362e+00 9.14441e+00
  4.08810e+04 1.00000e+00 1.00000e+00]]


In [16]:
#for high prediction
y_pred_h = regressor.predict(last_row) # test the output by changing values
print('high prediction is:', y_pred_h)

high prediction is: [1.1323243]


In [17]:
# #for close prediction
# # Visualising the Random Forest Regression results

# # arange for creating a range of values
# # from min value of x to max
# # value of x with a difference of 0.01
# # between two consecutive values
# X_grid = np.arange(min(x), max(x), 1)

# # reshape for reshaping the data into a len(X_grid)*1 array,
# # i.e. to make a column out of the X_grid value				
# X_grid = X_grid.reshape((len(X_grid), 1))

# # Scatter plot for original data
# plt.scatter(x, y, color = 'blue')

# # plot predicted data
# plt.scatter(X_grid, regressor.predict(X_grid),
# 		color = 'green')
# plt.title('Random Forest Regression')
# plt.xlabel('Position level')
# plt.ylabel('EURUSD')
# plt.xlim(1900,1994)
# plt.ylim(1.13,1.17)
# plt.show()


In [18]:
#for low prediction
#select rows to use for x,y
x = df_m_5.iloc[:,np.r_[1:34]].values
print(x)
y = df_m_5.iloc[:,[35]].values
print(y) 

[[0.00000e+00 1.38676e+00 1.39374e+00 ... 6.36990e+04 1.00000e+00
  1.00000e+00]
 [1.00000e+00 1.39062e+00 1.39474e+00 ... 4.66600e+04 1.00000e+00
  1.00000e+00]
 [2.00000e+00 1.39251e+00 1.39427e+00 ... 5.52010e+04 0.00000e+00
  1.00000e+00]
 ...
 [1.99400e+03 1.12467e+00 1.12551e+00 ... 3.66050e+04 0.00000e+00
  0.00000e+00]
 [1.99500e+03 1.11959e+00 1.12295e+00 ... 2.60930e+04 1.00000e+00
  1.00000e+00]
 [1.99600e+03 1.12058e+00 1.13295e+00 ... 4.08810e+04 1.00000e+00
  1.00000e+00]]
[[1.38788]
 [1.38797]
 [1.38095]
 ...
 [1.11959]
 [1.12051]
 [1.13052]]


In [19]:
#for low prediction
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)

In [20]:
#for low prediction and accuracy test
# create regressor object
regressor = xg.XGBRegressor(n_estimators=10000, objective='reg:squarederror', nthread=-1, n_jobs=-1)
#regressor = rf(n_estimators = 1000, n_jobs=-1, criterion="absolute_error", random_state=2)
# fit the regressor with x and y data
regressor.fit(X_train, y_train)

XGBRegressor(n_estimators=10000, n_jobs=-1, nthread=-1,
             objective='reg:squarederror')

In [21]:
#for low pred 
y_pred = regressor.predict(X_test)
mape = mean_absolute_percentage_error(y_test,y_pred)

In [22]:
#for low pred regression score
from sklearn import metrics 
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('MAPE:', mape)
print('MedAE', metrics.median_absolute_error(y_test,y_pred)) 

MAE: 0.003115888036155705
MSE: 1.7317060498577302e-05
RMSE: 0.004161377235793134
MAPE: 6.563024001412784
MedAE 0.0024142199897767558


In [23]:
#for low pred 
y_pred_l = regressor.predict(last_row) # test the output by changing values
print('low prediction is:', y_pred_l)

low prediction is: [1.1302643]


In [24]:
# #for low prediction
# # Visualising the Random Forest Regression results

# # arange for creating a range of values
# # from min value of x to max
# # value of x with a difference of 0.01
# # between two consecutive values
# X_grid = np.arange(min(x), max(x), 1)

# # reshape for reshaping the data into a len(X_grid)*1 array,
# # i.e. to make a column out of the X_grid value				
# X_grid = X_grid.reshape((len(X_grid), 1))

# # Scatter plot for original data
# plt.scatter(x, y, color = 'blue')

# # plot predicted data
# plt.scatter(X_grid, regressor.predict(X_grid),
# 		color = 'green')
# plt.title('Random Forest Regression')
# plt.xlabel('Position level')
# plt.ylabel('EURUSD')
# plt.xlim(7980,8000)
# plt.ylim(1.15,1.17)
# plt.show()

In [25]:
#for close prediction
#select rows to use for x,y
x = df_m_5.iloc[:,np.r_[1:34]].values
print(x)
y = df_m_5.iloc[:,[36]].values
print(y) 

[[0.00000e+00 1.38676e+00 1.39374e+00 ... 6.36990e+04 1.00000e+00
  1.00000e+00]
 [1.00000e+00 1.39062e+00 1.39474e+00 ... 4.66600e+04 1.00000e+00
  1.00000e+00]
 [2.00000e+00 1.39251e+00 1.39427e+00 ... 5.52010e+04 0.00000e+00
  1.00000e+00]
 ...
 [1.99400e+03 1.12467e+00 1.12551e+00 ... 3.66050e+04 0.00000e+00
  0.00000e+00]
 [1.99500e+03 1.11959e+00 1.12295e+00 ... 2.60930e+04 1.00000e+00
  1.00000e+00]
 [1.99600e+03 1.12058e+00 1.13295e+00 ... 4.08810e+04 1.00000e+00
  1.00000e+00]]
[[1.39252]
 [1.39304]
 [1.38223]
 ...
 [1.12062]
 [1.13177]
 [1.13097]]


In [26]:
#for close prediction
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)

In [27]:
#for close prediction and accuracy test

# create regressor object
regressor = xg.XGBRegressor(n_estimators=10000, objective='reg:squarederror', nthread=-1, n_jobs=-1)
#regressor = rf(n_estimators = 1000, n_jobs=-1, criterion="absolute_error", random_state=3)
# fit the regressor with x and y data
regressor.fit(X_train, y_train)

XGBRegressor(n_estimators=10000, n_jobs=-1, nthread=-1,
             objective='reg:squarederror')

In [28]:
#for close prediction
y_pred = regressor.predict(X_test)
mape = mean_absolute_percentage_error(y_test,y_pred)

In [29]:
#for close regression score
from sklearn import metrics 
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('MAPE:', mape)
print('MedAE', metrics.median_absolute_error(y_test,y_pred)) 
#print('MdAPE', metrics)

MAE: 0.004553578386497495
MSE: 3.6408061831196936e-05
RMSE: 0.006033909332364627
MAPE: 6.581921087253724
MedAE 0.0036106148529053694


In [30]:
#for close pred 
y_pred_c = regressor.predict(last_row) # test the output by changing values
print('close prediction is:', y_pred_c)

close prediction is: [1.1313605]


In [31]:
print('Today the predictions are:')
print('high', y_pred_h)
print('close', y_pred_c)
print('low', y_pred_l)
print('Please use these as polarities.')

Today the predictions are:
high [1.1323243]
close [1.1313605]
low [1.1302643]
Please use these as polarities.


In [32]:
# #for high prediction
# # Visualising the Random Forest Regression results

# # arange for creating a range of values
# # from min value of x to max
# # value of x with a difference of 0.01
# # between two consecutive values
# X_grid = np.arange(min(x), max(x), 1)

# # reshape for reshaping the data into a len(X_grid)*1 array,
# # i.e. to make a column out of the X_grid value				
# X_grid = X_grid.reshape((len(X_grid), 1))

# # Scatter plot for original data
# plt.scatter(x, y, color = 'blue')

# # plot predicted data
# plt.scatter(X_grid, regressor.predict(X_grid),
# 		color = 'green')
# plt.title('Random Forest Regression')
# plt.xlabel('Position level')
# plt.ylabel('EURUSD')
# plt.xlim(7980,8002)
# plt.ylim(1.15,1.17)
# plt.show()